In [1]:
import tensorflow as tf
from tensorflow.contrib.layers import xavier_initializer as xinit

In [7]:
import numpy as np

In [2]:
import gym

## build network

- input : state
- output : Q-values for each action

In [3]:
state_size = 4
action_size = 2
hdim = 24

In [5]:
class DQNAgent():
    
    def __init__(self, state_size=4, action_size=2, hdim=24, epsilon=1.):
        
        self.epsilon = epsilon
        self.action_size = action_size
        
        tf.reset_default_graph()
        self.state_ = tf.placeholder(tf.float32, shape=[None, state_size], name='states')
        self.target_ = tf.placeholder(tf.float32, shape=[None, action_size], name='targets')
        W1 = tf.get_variable('W1', dtype=tf.float32, shape=[state_size, hdim], initializer=xinit() )
        b1 = tf.get_variable('b1', dtype=tf.float32, shape=[hdim], initializer=xinit() )
        h = tf.nn.relu(tf.matmul(self.state_, W1) + b1)
        W2 = tf.get_variable('W2', dtype=tf.float32, shape=[hdim, action_size], initializer=xinit() )
        b2 = tf.get_variable('b2', dtype=tf.float32, shape=[action_size], initializer=xinit() )
        self.q_out = tf.matmul(h, W2) + b2
        self.loss = tf.reduce_mean(tf.pow(self.q_out - self.target_, 2))
        
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())
        

    def get_action(self, s):
        if np.random.rand() <= self.epsilon:
            return np.random.randint(0, self.action_size)
        
        q_values = self.sess.run(self.q_out, feed_dict = {
            self.state_ : s
        })[0]
        
        return np.argmax(q_values)